Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [2]:
#Ваш код здесь
# df['gender'].value_counts()
print('Female:', df[(df['gender'] == 'Female')].count()[0])
print('Male: ', df[(df['gender'] == 'Male')].count()[0])

Female: 3488
Male:  3555


##### 2. Какое количество уникальных значений у поля InternetService?

In [3]:
#Ваш код здесь
df['InternetService'].value_counts()

Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [4]:
#Ваш код здесь
df[['TotalCharges']].describe()

,TotalCharges
count,7043
unique,6531
top,20.2
freq,11


В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

In [5]:
# поле строкового типа 
# отобрать непустые числовые значения, посчитать статистики для них
total_charges = pd.to_numeric(df['TotalCharges'], errors='coerce')
total_charges = total_charges[~total_charges.isna()]
# len(total_charges)
print(f'median:{total_charges.median()}; mean:{total_charges.mean()}; std:{total_charges.std()}')

median:1397.475; mean:2283.3004408418697; std:2266.7713618831453


##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [6]:
#Ваш код здесь
encoded_phone_service = df['PhoneService'].apply(lambda v: v.replace('Yes', '1').replace('No', '0')).astype(int)
df['PhoneService'] = encoded_phone_service

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [7]:
def clean_charges_space(value):
    if value == ' ':
        return np.nan
    else:
        return value

spaced_charges = df['TotalCharges'].apply(clean_charges_space).astype(float)
spaced_charges.fillna(value = 0, inplace = True)
# len(spaced_charges)
df['TotalCharges'] = spaced_charges
print(f'median:{total_charges.median()}; mean:{total_charges.mean()}; std:{total_charges.std()}')

median:1397.475; mean:2283.3004408418697; std:2266.7713618831453


##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [8]:
#Ваш код здесь
encoded_churn = df['Churn'].apply(lambda v: v.replace('Yes', '1').replace('No', '0')).astype(int)
df['Churn'] = encoded_churn

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [9]:
#Ваш код здесь
encoded_streaming = df['StreamingMovies'].apply(lambda v: v.replace('Yes', '1').replace('No internet service', '0').replace('No', '0')).astype(int)
df['StreamingMovies'] = encoded_streaming
df['StreamingMovies'].value_counts()

0    4311
1    2732
Name: StreamingMovies, dtype: int64

In [10]:
encoded_streaming = df['StreamingTV'].apply(lambda v: v.replace('Yes', '1').replace('No internet service', '0').replace('No', '0')).astype(int)
df['StreamingTV'] = encoded_streaming
df['StreamingTV'].value_counts()

0    4336
1    2707
Name: StreamingTV, dtype: int64

In [11]:
encoded_tech = df['TechSupport'].apply(lambda v: v.replace('Yes', '1').replace('No internet service', '0').replace('No', '0')).astype(int)
df['TechSupport'] = encoded_tech
df['TechSupport'].value_counts()

0    4999
1    2044
Name: TechSupport, dtype: int64

##### 8. Заполните пропуски в поле PhoneService значением 0

In [12]:
#Ваш код здесь
df[df['PhoneService'].isna()]

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn


#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [13]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
#Ваш код здесь
df = df[columns]
df.head(3)

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,0,29.85,0,0,0,0
1,Male,34,1,1889.50,0,0,0,0
2,Male,2,1,108.15,0,0,0,1


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [14]:
from sklearn.model_selection import train_test_split

RANDOM_STATE = 21
features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'

#Ваш код здесь
train = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(train, y, 
                                                    shuffle=True,
                                                    test_size=0.3,
                                                    random_state=RANDOM_STATE,
                                                    stratify=y)

##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [15]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [16]:
#Ваш код здесь
gender.fit_transform(X_test)

,gender_Female,gender_Male
6470,0,1
6718,0,1
1253,1,0
1006,1,0
2867,1,0
...,...,...
6974,1,0
5363,0,1
1988,0,1
5453,0,1


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [17]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [18]:
#Ваш код здесь
TotalCharges =  Pipeline([
                ('selector', NumberSelector(key='TotalCharges')),
                ('standard', StandardScaler())
            ])

Объединение всех "кубиков" очень легко сделать таким образом

In [19]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [20]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [21]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('TotalCharges',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='TotalCharges')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
          

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [22]:
#Ваш код здесь
preds = pipeline.predict_proba(X_test)[:, 1]

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [23]:
from sklearn.metrics import roc_auc_score, log_loss
#Ваш код здесь
print (f'AUC&ROC: {roc_auc_score(y_test, preds)}')
print (f'log_loss: {roc_auc_score(y_test, preds)}')

AUC&ROC: 0.7508619778745612
log_loss: 0.7508619778745612


### Сохраним наш пайплайн

In [ ]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)